In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import time
import csv
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.animation as animation
import matplotlib as mpl
from scipy import interpolate
from scipy import ndimage
#import numpy as np, cv2
import matplotlib.patches as patches

mpl.rcParams['axes.linewidth'] = 3.0 #set the value globally
#mpl.use("Agg")

In [3]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [4]:
import sys 
sys.path.insert(0, '/home/martial/Data/programs/Hyperspy_ungit')

import hyperspy.api as hs

In [5]:
hs.hyperspy.Release.info

'\n    H y p e r S p y\n    Version 1.6.dev\n\n    http://www.hyperspy.org\n\n    '

In [6]:
def align2D_cv(hs_stack, warp_mode):
    
    hs_stack.change_dtype('float32')

    #shift
    warp_matrix_max = np.zeros(4)
    
    # High pass filter using low pass and Gaussian blurring
    filter_1 = ndimage.gaussian_filter(hs_stack.data[0], 10)
    
    # Convert Numpy hpf data in openCV format (3 channels)
    img_1_hpf_cv = cv2.cvtColor(filter_1, cv2.COLOR_GRAY2BGR)
    # Convert openCV format (3 channels) in openCV format grey (1 channel)
    img_1_hpf_cv = cv2.cvtColor(img_1_hpf_cv, cv2.COLOR_BGR2GRAY)
     
    #find the shape
    sz = img_1_hpf_cv.shape
    
    #Translation ( MOTION_TRANSLATION ) : The first image can be shifted ( translated ) by (x , y) to obtain the second image. There are only two parameters x and y that we need to estimate.
    #Euclidean ( MOTION_EUCLIDEAN ) : The first image is a rotated and shifted version of the second image. So there are three parameters — x, y and angle . You will notice in Figure 4, when a square undergoes Euclidean transformation, the size does not change, parallel lines remain parallel, and right angles remain unchanged after transformation.
    #Affine ( MOTION_AFFINE ) : An affine transform is a combination of rotation, translation ( shift ), scale, and shear. This transform has six parameters. When a square undergoes an Affine transformation, parallel lines remain parallel, but lines meeting at right angles no longer remain orthogonal.
    #Homography ( MOTION_HOMOGRAPHY ) : All the transforms described above are 2D transforms. They do not account for 3D effects. A homography transform on the other hand can account for some 3D effects ( but not all ). This transform has 8 parameters. A square when transformed using a Homography can change to any quadrilateral.
    
    #warp_mode = cv2.MOTION_TRANSLATION
    
    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        
    warp_matrix_returned = np.empty((hs_stack.data.shape[0], warp_matrix.shape[0], warp_matrix.shape[1]))
    
    # Specify the number of iterations.
    number_of_iterations = 3000;
    
    # Specify the threshold of the increment
    termination_eps = 1e-10;
    
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    for i in range (1,hs_stack.data.shape[0]):
        filter_2 = ndimage.gaussian_filter(hs_stack.data[i], 10)
        # Convert Numpy hpf data in openCV format (3 channels)
        img_2_hpf_cv = cv2.cvtColor(filter_2, cv2.COLOR_GRAY2BGR)
        # Convert openCV format (3 channels) in openCV format grey (1 channel)
        img_2_hpf_cv = cv2.cvtColor(img_2_hpf_cv, cv2.COLOR_BGR2GRAY)
    
        # Run the ECC algorithm. The results are stored in warp_matrix.
        (cc, warp_matrix) = cv2.findTransformECC(img_1_hpf_cv,img_2_hpf_cv,warp_matrix, warp_mode, criteria)
    
        warp_matrix_returned[i,:,:] = warp_matrix
    
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            # Use warpPerspective for Homography 
            hs_stack.data[i] = cv2.warpPerspective (hs_stack.data[i], warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else :
            # Use warpAffine for Translation, Euclidean and Affine
            hs_stack.data[i] = cv2.warpAffine(hs_stack.data[i], warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP); 
           
        if warp_matrix[0,2]<warp_matrix_max[0]:
            warp_matrix_max[0]=warp_matrix[0,2]
            
        if warp_matrix[0,2]>warp_matrix_max[1]:
            warp_matrix_max[1]=warp_matrix[0,2]

        if warp_matrix[1,2]<warp_matrix_max[2]:
            warp_matrix_max[2]=warp_matrix[1,2]
            
        if warp_matrix[1,2]>warp_matrix_max[3]:
            warp_matrix_max[3]=warp_matrix[1,2]
            
        print(str(i) +' / '+str(hs_stack.data.shape[0]-1))
            
    hs_stack.crop_image(bottom=hs_stack.data.shape[2]-np.int(warp_matrix_max[3]), \
                        top = -np.int(warp_matrix_max[2]), \
                        left=-np.int(warp_matrix_max[0]), \
                        right = hs_stack.data.shape[1]-np.int(warp_matrix_max[1]))
    
    return warp_matrix_returned

In [7]:
def align2D_cv_only_align(hs_stack, warp_matrix, warp_mode):
    
    hs_stack.change_dtype('float32')

    #shift
    warp_matrix_max = np.zeros(4)
       
    #find the shape
    sz = hs_stack.data[0].shape
        
    for i in range (1,hs_stack.data.shape[0]):
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            # Use warpPerspective for Homography 
            hs_stack.data[i] = cv2.warpPerspective (hs_stack.data[i], warp_matrix[i,:,:], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else :
            # Use warpAffine for Translation, Euclidean and Affine
            hs_stack.data[i] = cv2.warpAffine(hs_stack.data[i], warp_matrix[i,:,:], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP); 
           
        if warp_matrix[i,0,2]<warp_matrix_max[0]:
            warp_matrix_max[0]=warp_matrix[i,0,2]
            
        if warp_matrix[i,0,2]>warp_matrix_max[1]:
            warp_matrix_max[1]=warp_matrix[i,0,2]

        if warp_matrix[i,1,2]<warp_matrix_max[2]:
            warp_matrix_max[2]=warp_matrix[i,1,2]
            
        if warp_matrix[i,1,2]>warp_matrix_max[3]:
            warp_matrix_max[3]=warp_matrix[i,1,2]
            
        print(str(i) +' / '+str(hs_stack.data.shape[0]-1))
 
    hs_stack.crop_image(bottom=hs_stack.data.shape[2]-np.int(warp_matrix_max[3]), \
                        top = -np.int(warp_matrix_max[2]), \
                        left=-np.int(warp_matrix_max[0]), \
                        right = hs_stack.data.shape[1]-np.int(warp_matrix_max[1]))
       
    return

### Read voltage Thierry's file with the voltage in the 2nd column'

In [8]:
with open('Voltage_RGA/Time_voltage.csv', newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        line_count = line_count +1
        
Time_voltage = np.zeros((csv_reader.line_num, 3))

In [9]:
with open('Voltage_RGA/Time_voltage.csv', newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            Time_voltage[line_count,1]=float(''.join(row))
        line_count = line_count +1

### Loop on the images, read the time stamp and insert it in the 'Time_voltage' array

In [10]:
files_in_dir = [ f for f in listdir('Voltage_RGA//Data') if isfile(join('Voltage_RGA/Data',f)) ]
files_in_dir.sort()
len(files_in_dir)

1573

In [11]:
line_count = 0
for i in files_in_dir:
    Im = Image.open('Voltage_RGA/Data/'+i)
    try:
        time_ = Im._getexif()[36867][11:19]
        t = time.strptime(time_, '%H:%M:%S')
        Time_voltage[line_count,0]=(t.tm_hour*3600+t.tm_min*60+t.tm_sec)
    except:
        Time_voltage[line_count,0]=Time_voltage[line_count-1,0]+2
        Im.close()

    line_count = line_count +1

In [12]:
Time_voltage[2,1]=Time_voltage[1,1]

In [13]:
Time_voltage[0,0]

69039.0

In [14]:
for i in range (0,Time_voltage[:,1].size):
    if Time_voltage[i,1]<0.45:
        Time_voltage[i,1]=mean(Time_voltage[i-10::i-1,1])

Time_voltage[:,2] = (Time_voltage[:,1]-min(Time_voltage[:,1]))/(max(Time_voltage[:,1])-min(Time_voltage[:,1]))

### RGA Data

In [15]:
df = pd.read_csv('Voltage_RGA/16)_RGA_20190424  N2-H2_O2_Quentin_2019-04-23.csv')

In [16]:
RGA_data = np.zeros((df.shape[0], 3))

In [17]:
df.axes

[RangeIndex(start=0, stop=890, step=1),
 Index(['H2 Time [s]', 'H2 Time Relative [s]', 'H2 current [A]', 'O Time [s]',
        'O Time Relative [s]', 'O current [A]', 'N Time [s]',
        'N Time Relative [s]', 'N current [A]', 'N2 Time [s]',
        'N2 Time Relative [s]', 'N2 current [A]', 'O2 Time [s]',
        'O2 Time Relative [s]', 'O2 current [A]', 'Nox Time [s]',
        'Nox Time Relative [s]', 'Nox current [A]', 'H2O Time [s]',
        'H2O Time Relative [s]', 'H2O Time [A]'],
       dtype='object')]

In [18]:
for i in range(0,df.shape[0]):
    RGA_data[i,0] = time.strptime((df['O2 Time [s]'][i][11:19]), '%H:%M:%S').tm_sec + \
                    time.strptime((df['O2 Time [s]'][i][11:19]), '%H:%M:%S').tm_min*60 + \
                    time.strptime((df['O2 Time [s]'][i][11:19]), '%H:%M:%S').tm_hour*3600
RGA_data[:,1] = df['O2 current [A]']
RGA_data[:,2] = df['H2 current [A]']

RGA_data[:,1] = RGA_data[:,1]-np.min(RGA_data[:,1])
RGA_data[:,2] = RGA_data[:,2]-np.min(RGA_data[:,2])

### Plot time versus voltage

In [19]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('Time [min]')
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax1.plot(Time_voltage[:,0]-Time_voltage[0,0],Time_voltage[:,1], 'r+')
ax1.set_ylabel('Voltage [mV]')

ax2.plot(RGA_data[:,0]-Time_voltage[0,0],RGA_data[:,1], 'o', label='O2')
ax2.plot(RGA_data[:,0]-Time_voltage[0,0],RGA_data[:,2], 'v', label='H2')
ax2.set_ylabel('Current [ions]')
fig.tight_layout()

plt.show()

savetxt('time_voltage.txt', Time_voltage)

savetxt('RGA_data.txt', RGA_data)

### Read TEM images

In [21]:
TEM2 = hs.load('16)_380K_aligned_bin2.hdf5')

In [ ]:
#TEM2.save('16)_380K_aligned_bin2.rpl')

In [22]:
TEM2.metadata.General.time

'19:10:36'

In [23]:
time.strptime((TEM2.metadata.General.time), '%H:%M:%S').tm_sec+ \
    time.strptime((TEM2.metadata.General.time), '%H:%M:%S').tm_min*60 + \
    time.strptime((TEM2.metadata.General.time), '%H:%M:%S').tm_hour*3600

69036

In [24]:
RGA_data[7,0]

69055.0

In [25]:
Time_voltage[8,0] #Time_voltage[0,0] =69,039

69056.0

In [26]:
Tag_TEM_image = np.zeros((TEM2.data.shape[0], 6))
#Initial time limited by Time_voltage @ 69,039s
# column 0: time
# column 1: voltage
# column 2: O2 RGA
# column 3: TEM image_nbre
# column 4: extrapolated voltage
# column 5: H2 RGA

In [27]:
Tag_TEM_image[0,0]= Time_voltage[8,0] #corresponds to 69,056s (computer time)
Tag_TEM_image[0,1]= Time_voltage[8,1] #Voltage
Tag_TEM_image[0,2]= RGA_data[7,1] #RGA O2
Tag_TEM_image[0,3]= 1 #image nbre
Tag_TEM_image[0,5]= RGA_data[7,2] #RGA O2

i=0 #loop on RGA
k=0 #loop on voltage
for j in range(1,TEM2.data.shape[0]):
    while (Tag_TEM_image[j-1,0]+20)>RGA_data[i,0]  and (i<RGA_data.data.shape[0]-1):
        i=i+1
    
    while ((Tag_TEM_image[j-1,0]+20)>Time_voltage[k,0]) and (k<Time_voltage.data.shape[0]-1):
        k=k+1
    
    Tag_TEM_image[j,0]= Time_voltage[k,0] #Time
    Tag_TEM_image[j,1]= Time_voltage[k,1] #Voltage
    Tag_TEM_image[j,2]= RGA_data[i,1] #RGA O2
    Tag_TEM_image[j,3]= j
    Tag_TEM_image[j,5]= RGA_data[i,2] #RGA H2
    
Tag_TEM_image[:,0]= Tag_TEM_image[:,0] - Tag_TEM_image[0,0] #Time

In [28]:
tck = interpolate.splrep(Tag_TEM_image[:,0], Tag_TEM_image[:,1], s=0.1 )
Tag_TEM_image[:,4] = interpolate.splev(Tag_TEM_image[:,0], tck)

In [29]:
mpl.rcParams['axes.linewidth'] = 3.5 #set the value globally

fig, axes = plt.subplots(sharex=False, sharey=False, squeeze=True)
fig.set_tight_layout(True)

# x axis
axes.set_xlabel('Time [min]',fontsize=20)
ax2 = axes.twinx()  # instantiate a second axes that shares the same x-axis

# Voltage graph
axes.plot(Tag_TEM_image[:,0]/60,Tag_TEM_image[:,1], color='red')
axes.plot(Tag_TEM_image[:,0]/60, Tag_TEM_image[:,4], lw=5, color='red')
axes.set_ylabel('Voltage [mV]',fontsize=20)
axes.set_xlim(0, TEM2.data.shape[0]*20/60)
axes.set_ylim(np.min(Tag_TEM_image[:,1])*0.8, np.max(Tag_TEM_image[:,1])*1.2)

# O2/H2 current graph
ax2.plot(Tag_TEM_image[:,0]/60-4,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
         - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
ax2.plot(Tag_TEM_image[:,0]/60,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
         - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
ax2.set_ylabel('O$_2$/H$_2$ current ratio (x100)',fontsize=20)
ax2.set_ylim(100*(np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] ))), \
                100*(np.max(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])))*1.1)

plt.setp(axes.get_xticklabels(), fontsize=17)
plt.setp(axes.get_yticklabels(), fontsize=17)
plt.setp(ax2.get_yticklabels(), fontsize=17)
ax2.yaxis.set_ticks(np.arange(0, 3.99, 1))

fig.tight_layout()
fig.savefig("Day2_exp16.png", dpi=300)
plt.show()

### Video

In [ ]:
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib', comment='Movie support!')
writer = FFMpegWriter(fps=5, metadata=metadata)

In [ ]:
mpl.rcParams['axes.linewidth'] = 3.5 #set the value globally

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, squeeze=True, \
                         figsize=(12,6), gridspec_kw={'width_ratios': [1.2, 1]})
fig.set_tight_layout(True)
plt.setp(axes[0], xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = axes[0].imshow(TEM2.data[1], cmap='gray', vmin=0, vmax=3*np.mean(TEM2.data[1]))

# x axis
axes[1].set_xlabel('Time [min]',fontsize=20)
ax2 = axes[1].twinx()  # instantiate a second axes that shares the same x-axis

# Voltage graph
#axes[1].plot(Tag_TEM_image[:,0]/60,Tag_TEM_image[:,1], color='red')
#axes[1].plot(Tag_TEM_image[:,0]/60, Tag_TEM_image[:,4], lw=5, color='red')
axes[1].set_ylabel('Voltage [mV]',fontsize=20)
axes[1].set_xlim(0, TEM2.data.shape[0]*20/60)
axes[1].set_ylim(np.min(Tag_TEM_image[:,1])*0.8, np.max(Tag_TEM_image[:,1])*1.2)

# O2/H2 current graph
#ax2.plot(Tag_TEM_image[:,0]/60-4,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
#         - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
#ax2.plot(Tag_TEM_image[:,0]/60,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
#        - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
ax2.set_ylabel('O$_2$/H$_2$ current ratio (x100)',fontsize=20)

ax2.set_ylim(100*(np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] ))), \
                100*(np.max(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5]))*1.1))

plt.setp(axes[1].get_xticklabels(), fontsize=17)
plt.setp(axes[1].get_yticklabels(), fontsize=17)
plt.setp(ax2.get_yticklabels(), fontsize=17)
ax2.yaxis.set_ticks(np.arange(0, 4.5, 1))

fig.tight_layout()

plt.show()

In [ ]:
with writer.saving(fig, "Day2_exp16.mp4", dpi=300):
    for i in range(1,TEM2.data.shape[0],1):
        im = axes[0].imshow(TEM2.data[i], cmap='gray', vmin=0, vmax=2*np.mean(TEM2.data[i]))
        im = axes[1].scatter(Tag_TEM_image[:int(i),0]/60, Tag_TEM_image[:int(i),1], color='red')
        im = axes[1].plot(Tag_TEM_image[:int(i),0]/60, Tag_TEM_image[:int(i),4], lw=5, color='red')
                #Shift of 4 minutes correspond to ~12 i
        shift_i = 12
        im = ax2.plot(Tag_TEM_image[:int(i),0]/60,100*(Tag_TEM_image[shift_i:int(i+shift_i),2] \
            /Tag_TEM_image[shift_i:int(i+shift_i),5] \
             - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
        im = ax2.plot(Tag_TEM_image[:int(i),0]/60,100*(Tag_TEM_image[:int(i),2]/Tag_TEM_image[:int(i),5] \
            - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
        
        textstr = '\n'.join((
            'Img nbr: ' + str(i),
            'Time: ' + str(round(20*Tag_TEM_image[(i),3]/60,2)) +' min',
            str(round(Tag_TEM_image[(i),1],2)) + '  V'))
            #str(round(100*(Tag_TEM_image[(i),2]/Tag_TEM_image[(i),5]- np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])) \
            #          ,1))+ ' O$_2$/H$_2$ current ratio (x100)'))
        
        textbox = plt.text(20*40./60, 0.8, textstr, fontsize=12, verticalalignment='top', \
                           backgroundcolor='white')   
        writer.grab_frame()
        textbox.remove()
        print(str(i) + ' over ' + str(TEM2.data.shape[0]-1))

### Binned

In [ ]:
TEM_bin4 = hs.load('16)_380K_bin4_ali.dm4')

In [ ]:
TEM_bin4.crop_image(top=80., left=20., bottom=115., right=100.)

In [ ]:
warp_matrix_returned_bin4 = align2D_cv(TEM_bin4, cv2.MOTION_TRANSLATION)

In [ ]:
warp_matrix_returned_bin4 = np.load('warp_matrix_returned_bin4.npy')

np.save('warp_matrix_returned_bin4.npy',warp_matrix_returned_bin4)

In [ ]:
TEM_bin4 = hs.load('16)_380K_bin4_ali.dm4')

In [ ]:
align2D_cv_only_align(TEM_bin4, warp_matrix_returned_bin4, cv2.MOTION_TRANSLATION)

In [ ]:
TEM_bin4.plot()

In [ ]:
warp_matrix_returned_bin2 = np.empty_like(warp_matrix_returned_bin4)
warp_matrix_returned_bin2 = warp_matrix_returned_bin4
warp_matrix_returned_bin2[:,0,2] = 2*warp_matrix_returned_bin4[:,0,2]
warp_matrix_returned_bin2[:,1,2] = 2*warp_matrix_returned_bin4[:,1,2]

In [ ]:
TEM_bin2 = hs.load('16)_380K_bin2_ali.dm4')

align2D_cv_only_align(TEM_bin2, warp_matrix_returned_bin2, cv2.MOTION_TRANSLATION)

In [ ]:
TEM_bin2.plot()

In [ ]:
TEM_bin2.save('TEM_bin2_aligned.hdf5')

In [ ]:
TEM_bin2.plot()

# Not binned

In [ ]:
TEM = hs.load('16)_380K_crop.hdf5')

In [ ]:
warp_matrix_returned = align2D_cv(TEM, cv2.MOTION_TRANSLATION)

In [ ]:
np.save('warp_matrix_returned.npy', warp_matrix_returned)

In [ ]:
TEM.plot()

### FFT videos

In [7]:
TEM_croptr = hs.load('Extract_16)_380K_aligned_bin2_tr.dm4')

In [ ]:
TEM_croptr.plot()

In [8]:
TEM_croptr_fft = TEM_croptr.fft(shift=True, apodization=False)

In [9]:
TEM_croptr_fft.data = np.log(np.abs(TEM_croptr_fft.data))

In [31]:
TEM_cropbl = hs.load('Extract_16)_380K_aligned_bin2_bl.dm4')

In [32]:
TEM_cropbl.plot()

In [11]:
TEM_cropbl_fft = TEM_cropbl.fft(shift=True, apodization=False)

In [12]:
TEM_cropbl_fft.data = np.log(np.abs(TEM_cropbl_fft.data))

### Save images

In [4]:
TEM2 = hs.load('16)_380K_aligned_bin2.hdf5')

img_nbr=0
grid = GridSpec(nrows=2, ncols=4, left=0.1, bottom=0.235, right=0.9, top=0.765, wspace=0.05, hspace=0.05)

fig = plt.figure(0)
fig.clf()
rect1 = patches.Rectangle((912,149),512,512,linewidth=4,edgecolor='orange',facecolor='none', linestyle = '--')
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='purple',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0:2, 0:2])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect1)
ax1.add_patch(rect2)

ax20 = fig.add_subplot(grid[0, 2])
plt.setp(ax20, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax20.spines[pos].set_edgecolor('orange')
im = ax20.imshow(TEM_croptr.data[img_nbr], cmap='gray', \
                vmin=0, \
                vmax=2.7*np.mean(TEM_croptr.data[img_nbr]))

ax21 = fig.add_subplot(grid[0, 3])
plt.setp(ax21, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax21.spines[pos].set_edgecolor('orange')
im = ax21.imshow(TEM_croptr_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[img_nbr]))
ax21.arrow(354, 46, -17.5, 38, head_width=20., color='orange')
  
ax30 = fig.add_subplot(grid[1, 2])
plt.setp(ax30, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax30.spines[pos].set_edgecolor('yellow')
im = ax30.imshow(TEM_cropbl.data[img_nbr], cmap='gray', \
                vmin=0, \
                vmax=2.7*np.mean(TEM_cropbl.data[img_nbr]))
    
ax31 = fig.add_subplot(grid[1, 3])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('yellow')
im = ax31.imshow(TEM_cropbl_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[img_nbr]))
ax31.arrow(129, 47, -17.5/2, 38/2, head_width=20./2, color='purple')
ax31.arrow(108, 23, -17.5/2, 38/2, head_width=20./2, color='orange')

fig.savefig('TEM_exp16_nbr'+str(img_nbr)+'.png', dpi=600)

#### IMG+FFT

In [30]:
img_nbr=107

#fig = plt.figure(0)
fig, axes = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, squeeze=True, \
                         figsize=(8,6))#, gridspec_kw={'width_ratios': [1.2, 1]})
grid = GridSpec(nrows=2, ncols=3, left=0.125, bottom=0.15, right=0.88, top=0.85, wspace=0.05, hspace=0.05)

fig.clf()
rect1 = patches.Rectangle((912,149),512,512,linewidth=4,edgecolor='orange',facecolor='none', linestyle = '--')
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='purple',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0:2, 0:2])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect1)
ax1.add_patch(rect2)

ax21 = fig.add_subplot(grid[0, 2])
plt.setp(ax21, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax21.spines[pos].set_edgecolor('orange')
im = ax21.imshow(TEM_croptr_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[img_nbr]))
ax21.arrow(354, 46, -17.5, 38, head_width=20., color='orange')
    
ax31 = fig.add_subplot(grid[1, 2])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('purple')
im = ax31.imshow(TEM_cropbl_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[img_nbr]))
ax31.arrow(129, 47, -17.5/2, 38/2, head_width=20./2, color='purple')
ax31.arrow(108, 23, -17.5/2, 38/2, head_width=20./2, color='orange')

fig.savefig('TEM_exp16_nbr_small_'+str(img_nbr)+'.png', dpi=600)

NameError: name 'TEM_croptr_fft' is not defined

#### IMG no FFT

In [87]:
img_nbr=19

#fig = plt.figure(0)
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, squeeze=True, \
                         figsize=(8,6), gridspec_kw={'width_ratios': [5, 1]})
grid = GridSpec(nrows=1, ncols=2, left=0.1, bottom=0.25, right=0.9, top=0.75, wspace=0.01, hspace=0.01)

fig.clf()
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='purple',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0, 0])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect2)
   
ax31 = fig.add_subplot(grid[0, 1])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('purple')
im = ax31.imshow(TEM_cropbl.data[img_nbr], cmap='gray', \
                vmin=0.4*np.mean(TEM_cropbl.data[img_nbr]), \
                vmax=2*np.mean(TEM_cropbl.data[img_nbr]))

fig.savefig('TEM_exp16_bl_'+str(img_nbr)+'.png', dpi=600)

### Video FFT+graph

### Video

In [366]:
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib', comment='Movie support!')
writer = FFMpegWriter(fps=5, metadata=metadata)

In [367]:
mpl.rcParams['axes.linewidth'] = 3.5 #set the value globally

grid = GridSpec(nrows=3, ncols=3, left=0.125, bottom=0.1, right=0.9, top=0.95, wspace=0.05, hspace=0.1)
fig = plt.figure(figsize=(6,6))
fig.clf()

rect1 = patches.Rectangle((912,149),512,512,linewidth=4,edgecolor='orange',facecolor='none', linestyle = '--')
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='yellow',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0:2, 0:2])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect1)
ax1.add_patch(rect2)

ax21 = fig.add_subplot(grid[0, 2])
plt.setp(ax21, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax21.spines[pos].set_edgecolor('orange')
im = ax21.imshow(TEM_croptr_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[img_nbr]))
ax21.arrow(354, 46, -17.5, 38, head_width=20., color='orange')
    
ax31 = fig.add_subplot(grid[1, 2])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('yellow')
im = ax31.imshow(TEM_cropbl_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[img_nbr]))
ax31.arrow(129, 47, -17.5/2, 38/2, head_width=20./2, color='yellow')
ax31.arrow(108, 23, -17.5/2, 38/2, head_width=20./2, color='orange')

# x axis graph
ax4 = fig.add_subplot(grid[2:3, 0:3])
ax4.set_xlabel('Time [min]',fontsize=10)
ax5 = ax4.twinx()  # instantiate a second axes that shares the same x-axis

# Voltage graph
#ax4.plot(Tag_TEM_image[:,0]/60,Tag_TEM_image[:,1], color='red')
#ax4.plot(Tag_TEM_image[:,0]/60, Tag_TEM_image[:,4], lw=5, color='red')
ax4.set_ylabel('Voltage [mV]',fontsize=10)
ax4.set_xlim(0, TEM2.data.shape[0]*20/60)
ax4.set_ylim(np.min(Tag_TEM_image[:,1])*0.8, np.max(Tag_TEM_image[:,1])*1.2)

# O2/H2 current graph
#ax5.plot(Tag_TEM_image[:,0]/60-4,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
#         - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
#ax5.plot(Tag_TEM_image[:,0]/60,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
#        - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
ax5.set_ylabel('O$_2$/H$_2$ current ratio (x100)',fontsize=10)

ax5.set_ylim(100*(np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] ))), \
                100*(np.max(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5]))*1.1))

plt.setp(ax4.get_xticklabels(), fontsize=7)
plt.setp(ax4.get_yticklabels(), fontsize=7)
plt.setp(ax5.get_yticklabels(), fontsize=7)
ax5.yaxis.set_ticks(np.arange(0, 4.5, 1))

plt.plot()

[]

In [368]:
with writer.saving(fig, "Day2_exp16_FFT.mp4", dpi=300):
    for i in range(1,TEM2.data.shape[0],1):
        im = ax1.imshow(TEM2.data[i], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
        im = ax21.imshow(TEM_croptr_fft.data[i], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[i]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[i]))
        im = ax31.imshow(TEM_cropbl_fft.data[i], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[i]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[i]))
        im = ax4.plot(Tag_TEM_image[:i,0]/60,Tag_TEM_image[:i,1], color='red')
        im = ax4.plot(Tag_TEM_image[:i,0]/60, Tag_TEM_image[:i,4], lw=5, color='red')
        im = ax5.plot(Tag_TEM_image[:i,0]/60-4,100*(Tag_TEM_image[:i,2]/Tag_TEM_image[:i,5] \
                 - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
        im = ax5.plot(Tag_TEM_image[:i,0]/60,100*(Tag_TEM_image[:i,2]/Tag_TEM_image[:i,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
        
        textstr = '\n'.join((
            'Img nbr: ' + str(i),
            'Time: ' + str(round(20*Tag_TEM_image[(i),3]/60,2)) +' min',
            str(round(Tag_TEM_image[(i),1],2)) + '  V'))
       
        textbox = plt.text(20*40./60, 1, textstr, fontsize=7, verticalalignment='top', \
                           backgroundcolor='white')   
        writer.grab_frame()
        textbox.remove()
        print(str(i) + ' over ' + str(TEM2.data.shape[0]-1))

1 over 160
2 over 160
3 over 160
4 over 160
5 over 160
6 over 160
7 over 160
8 over 160
9 over 160
10 over 160
11 over 160
12 over 160
13 over 160
14 over 160
15 over 160
16 over 160
17 over 160
18 over 160
19 over 160
20 over 160
21 over 160
22 over 160
23 over 160
24 over 160
25 over 160
26 over 160
27 over 160
28 over 160
29 over 160
30 over 160
31 over 160
32 over 160
33 over 160
34 over 160
35 over 160
36 over 160
37 over 160
38 over 160
39 over 160
40 over 160
41 over 160
42 over 160
43 over 160
44 over 160
45 over 160
46 over 160
47 over 160
48 over 160
49 over 160
50 over 160
51 over 160
52 over 160
53 over 160
54 over 160
55 over 160
56 over 160
57 over 160
58 over 160
59 over 160
60 over 160
61 over 160
62 over 160
63 over 160
64 over 160
65 over 160
66 over 160
67 over 160
68 over 160
69 over 160
70 over 160
71 over 160
72 over 160
73 over 160
74 over 160
75 over 160
76 over 160
77 over 160
78 over 160
79 over 160
80 over 160
81 over 160
82 over 160
83 over 160
84 over 160
8